In this document, we provide a brief tutorial on using CoinMarketCap Pro API to access cryptocurrency data. We'll display how to access exchange trading volume and cryptocurrency OHLCV data, but using other endpoints are essentially similar.

The only dependencies necessary to run the illustration are `load_dotenv` and `requests`.

In [13]:
import os
import requests

from dotenv import load_dotenv
load_dotenv()

True

Define a class which will contain the logic for accessing the API. The user can either pass `api_key` parameter directly to the `__init__` method or set it as an environment variable. The implementation supports `.env` files as `load_dotenv` has been called. Default base URL is automatically selected based on the existence of a paid API key, but can be overridden if the parameter is set by the user.

In [11]:
class CMC(object):

    _session = None
    __DEFAULT_BASE_URL = 'https://api.coinmarketcap.com/'
    __PRO_BASE_URL = 'https://pro-api.coinmarketcap.com/'
    __DEFAULT_TIMEOUT = 30
    __API_KEY_ENV_KEY = "CMC_PRO_API_KEY"

    def __init__(self, api_key=None, base_url=None, request_timeout=__DEFAULT_TIMEOUT):
        self.api_key = api_key or os.getenv(self.__API_KEY_ENV_KEY)
        if (not base_url):
            base_url = self.__PRO_BASE_URL if self.api_key \
                else self.__DEFAULT_BASE_URL

        self.base_url = base_url
        self.request_timeout = request_timeout

Add the `session` property, which persists the API key across all requests. Furthermore, a `Session` instance uses a connection pool and reuses the TCP connection, which will lead to a significant increase in performance.

In [5]:
class CMC(object):

    _session = None
    __DEFAULT_BASE_URL = 'https://api.coinmarketcap.com/'
    __PRO_BASE_URL = 'https://pro-api.coinmarketcap.com/'
    __DEFAULT_TIMEOUT = 30
    __API_KEY_ENV_KEY = "CMC_PRO_API_KEY"

    def __init__(self, api_key=None, base_url=None, request_timeout=__DEFAULT_TIMEOUT):
        self.api_key = api_key or os.getenv(self.__API_KEY_ENV_KEY)
        if (not base_url):
            base_url = self.__PRO_BASE_URL if self.api_key \
                else self.__DEFAULT_BASE_URL

        self.base_url = base_url
        self.request_timeout = request_timeout

    @property
    def session(self):
        if (self._session):
            return self._session
        self._session = requests.Session()
        self._session.headers.update({'Content-Type': 'application/json'})
        if (self.api_key):
            self._session.headers.update({"X-CMC_PRO_API_KEY": self.api_key})
        return self._session

Define a private method, `__make_req`, which will be used by public methods to send requests to the API. `__make_req` receives the endpoint address, a dictionary of parameters, which must comply the format explained in the endoint's documentation, and the version of the endpoint. If the API returns an error, it'll be automatically raised. Otherwise, the response will be converted to a `dict` object and returned.

In [6]:
class CMC(object):

    _session = None
    __DEFAULT_BASE_URL = 'https://api.coinmarketcap.com/'
    __PRO_BASE_URL = 'https://pro-api.coinmarketcap.com/'
    __DEFAULT_TIMEOUT = 30
    __API_KEY_ENV_KEY = "CMC_PRO_API_KEY"

    def __init__(self, api_key=None, base_url=None, request_timeout=__DEFAULT_TIMEOUT):
        self.api_key = api_key or os.getenv(self.__API_KEY_ENV_KEY)
        if (not base_url):
            base_url = self.__PRO_BASE_URL if self.api_key \
                else self.__DEFAULT_BASE_URL

        self.base_url = base_url
        self.request_timeout = request_timeout

    @property
    def session(self):
        if (self._session):
            return self._session
        self._session = requests.Session()
        self._session.headers.update({'Content-Type': 'application/json'})
        if (self.api_key):
            self._session.headers.update({"X-CMC_PRO_API_KEY": self.api_key})
        return self._session

    def __make_req(self, endpoint, params, v):
        if (self.api_key):
            params["CMC_PRO_API_KEY"] = self.api_key

        url = self.base_url + f"{v}/" + endpoint
        response_object = self.session.get(
            url, params=params, timeout=self.request_timeout)

        response_object.raise_for_status()

        return response_object.json()

Time to add the public methods. We'll define methods for cryptocurrency OHLCV and exchange quotes endpoints.

In [30]:
class CMC(object):

    _session = None
    __DEFAULT_BASE_URL = 'https://api.coinmarketcap.com/'
    __PRO_BASE_URL = 'https://pro-api.coinmarketcap.com/'
    __DEFAULT_TIMEOUT = 30
    __API_KEY_ENV_KEY = "CMC_PRO_API_KEY"

    def __init__(self, api_key=None, base_url=None, request_timeout=__DEFAULT_TIMEOUT):
        self.api_key = api_key or os.getenv(self.__API_KEY_ENV_KEY)
        if (not base_url):
            base_url = self.__PRO_BASE_URL if self.api_key \
                else self.__DEFAULT_BASE_URL

        self.base_url = base_url
        self.request_timeout = request_timeout

    @property
    def session(self):
        if (self._session):
            return self._session
        self._session = requests.Session()
        self._session.headers.update({'Content-Type': 'application/json'})
        if (self.api_key):
            self._session.headers.update({"X-CMC_PRO_API_KEY": self.api_key})
        return self._session

    def __make_req(self, endpoint, params, v):
        url = self.base_url + f"{v}/" + endpoint
        response_object = self.session.get(
            url, params=params, timeout=self.request_timeout)
        
        response_object.raise_for_status()

        return response_object.json()
    
    def ohlcv_historical(self, params):
        "https://coinmarketcap.com/api/documentation/v1/#operation/getV2CryptocurrencyOhlcvHistorical"
        return self.__make_req('cryptocurrency/ohlcv/historical', params, "v2")


    def quotes_historical(self, params):
        "https://coinmarketcap.com/api/documentation/v1/#operation/getV1ExchangeQuotesHistorical"
        return self.__make_req("exchange/quotes/historical", params, "v1")

The user can access the data now.

In [34]:
cmc = CMC()
cmc.quotes_historical({"slug": "kraken,binance", "count": "1"})

{'status': {'timestamp': '2022-09-06T14:55:37.113Z',
  'error_code': 0,
  'error_message': None,
  'elapsed': 22,
  'credit_count': 1,
  'notice': None},
 'data': {'binance': {'quotes': [{'timestamp': '2022-09-06T14:50:12.000Z',
     'quote': {'USD': {'volume_24h': 73522805073.14716,
       'timestamp': '2022-09-06T14:50:12.000Z'}},
     'num_market_pairs': 1702}],
   'id': 270,
   'name': 'Binance',
   'slug': 'binance'},
  'kraken': {'quotes': [{'timestamp': '2022-09-06T14:50:11.000Z',
     'quote': {'USD': {'volume_24h': 708319460.386547,
       'timestamp': '2022-09-06T14:50:11.000Z'}},
     'num_market_pairs': 619}],
   'id': 24,
   'name': 'Kraken',
   'slug': 'kraken'}}}

In [44]:
cmc.ohlcv_historical({"slug": "bitcoin,ethereum", "count": "2"})

{'status': {'timestamp': '2022-09-06T14:56:37.730Z',
  'error_code': 0,
  'error_message': None,
  'elapsed': 46,
  'credit_count': 1,
  'notice': None},
 'data': {'bitcoin': {'id': 1,
   'name': 'Bitcoin',
   'symbol': 'BTC',
   'quotes': [{'time_open': '2022-09-05T00:00:00.000Z',
     'time_close': '2022-09-05T23:59:59.999Z',
     'time_high': '2022-09-05T00:05:00.000Z',
     'time_low': '2022-09-05T11:33:00.000Z',
     'quote': {'USD': {'open': 19988.789818433175,
       'high': 20031.160251574245,
       'low': 19673.04733875358,
       'close': 19812.37147875423,
       'volume': 28813460025.44,
       'market_cap': 379258182345.46,
       'timestamp': '2022-09-05T23:59:59.999Z'}}}]},
  'ethereum': {'id': 1027,
   'name': 'Ethereum',
   'symbol': 'ETH',
   'quotes': [{'time_open': '2022-09-05T00:00:00.000Z',
     'time_close': '2022-09-05T23:59:59.999Z',
     'time_high': '2022-09-05T23:53:00.000Z',
     'time_low': '2022-09-05T10:49:00.000Z',
     'quote': {'USD': {'open': 1577.8